### Часть 1 и 2.

Датасет взят [отсюда](https://github.com/snkim/AutomaticKeyphraseExtraction/blob/master/Schutz2008.tar.gz). После обработки xml-файлов, процесс которой запечатлен в этом разделе, я получил тот датасет, который лежит в гитхабе (а именно, файлы ``.txt`` и ``.keywords``).

Собственную разметку ключевых слов я положил в тот же репозиторий, файлы с расширениями ``.mykeywords``.

Структура у ``.keywords`` и ``.mykeywords`` одинаковая: ключевые фразы разделены символом новой строки.

За эталон разметки принято пересечение множеств ключевых слов.

In [1]:
import os
import re
from bs4 import BeautifulSoup
import random

In [2]:
CURR = os.getcwd()
ROOT = 'Schutz2008'
TEXTS = 'FinalXML'
KEY_WORDS = 'quantitative-evaluation-dataset'
if 'my_dataset' not in os.listdir():
    os.mkdir('my_dataset')


raw_texts = os.listdir(os.path.join(CURR, ROOT, TEXTS))

def resub(x):
    x = re.sub('\xa0', ' ', x)
    return re.sub(' \[[^\]]+\]', '', x)

In [3]:
chosen_texts = random.sample(raw_texts, 5)

for title in chosen_texts:
    rtitle = title.rsplit('.', maxsplit=1)[0]
    
    ## text part
    path = os.path.join(CURR, ROOT, TEXTS, title)
    
    with open(path, 'r', encoding='utf-8') as f:
        fList = []
        lines = f.readlines()
        for line in lines:
            soup = BeautifulSoup(line, 'lxml')
            p_s = soup.find_all('p')
            if p_s:
                fList.extend(list(p_s))
        
    art_text = '\n'.join([resub(x.text).strip() for x in fList])
    
    with open(os.path.join(CURR, 'my_dataset', rtitle + '.txt'), 'w', encoding='utf-8') as fw:
        fw.write(art_text)
        
    ## keywords part
    ## tbn: I only use keywords that were assigned by authors or publishers;
    ## some of predicted keywords will be used as my own, though
    path = os.path.join(CURR, ROOT, KEY_WORDS, title)
    with open(path, 'r', encoding='utf-8') as f:
        kList = []
        
        ksoup = BeautifulSoup(f.read(), 'lxml')
        GS = ksoup.find('gold-standard')
        for term in GS.find_all('keyphrase'):
            kList.append(str(term.get('term')))
        
    
            
    with open(os.path.join(CURR, 'my_dataset', rtitle + '.keywords'), 'w', encoding='utf-8') as fk:
        fk.write('\n'.join(kList))

In [4]:
print(chosen_texts)
# output:
##[
## 'Pediatr_Radiol-4-1-2292494.xml',
## 'Evid_Based_Complement_Alternat_Med-4-1-1810371.xml',
## 'BMC_Struct_Biol-4-_-331416.xml',
## 'J_Chem_Ecol-4-1-2292484.xml', 
## 'Int_J_Cardiovasc_Imaging-4-1-2233708.xml'
##]

['Pediatr_Radiol-4-1-2292494.xml', 'Evid_Based_Complement_Alternat_Med-4-1-1810371.xml', 'BMC_Struct_Biol-4-_-331416.xml', 'J_Chem_Ecol-4-1-2292484.xml', 'Int_J_Cardiovasc_Imaging-4-1-2233708.xml']


In [11]:
import RAKE
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize

lemmatizer = WordNetLemmatizer()
rake = RAKE.Rake(stopwords.words('english'))

MD = 'my_dataset'

In [12]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def normText(text):
    sents = []
    for sent in sent_tokenize(text):
        lemmas = []
        for t in nltk.word_tokenize(sent):
            lemmas.append(lemmatizer.lemmatize(t.lower(), get_wordnet_pos(t)))
        sents.append(' '.join(lemmas))
    return '.'.join(sents)

In [13]:
kw_files = [x for x in os.listdir('./my_dataset') if x.endswith('keywords')]
#print(kw_files)

keyDict = {}
papers = {}

for ftitle in kw_files:
    kDkey = ftitle.rsplit('.', maxsplit=1)[0]
    with open(os.path.join(CURR, MD, ftitle), 'r', encoding='utf-8') as fkw:
        kList = [re.sub('\xa0', ' ', x.strip()) for x in fkw.readlines()]
        if kDkey in keyDict:
            keyDict[kDkey].extend(kList)
        else:
            keyDict[kDkey] = kList
            
for k in keyDict:
    papers[k] = {}
    papers[k]['kw'] = list(set(keyDict[k]))
    
txt_files = [x for x in os.listdir('./my_dataset') if x.endswith('txt')]

for txt in txt_files:
    with open(
        os.path.join(CURR, MD, txt), 'r', encoding='utf-8'
    ) as f:
        raw_text = f.read()
    papers[txt.rsplit('.', maxsplit=1)[0]]['text'] = normText(raw_text)
    
print(len(papers))

5


### Часть 3

In [14]:
# rake keywords
rake_keywords = {}

for k in papers:
    text = papers[k]['text']
    rake_keywords[k] = rake.run(text, maxWords=3, minFrequency=2)
    
print(rake_keywords)

{'BMC_Struct_Biol-4-_-331416': [('ψ-blast/quest search strategy', 17.984848484848484), ('small c-terminal domain', 11.643120393120393), ('α-carbon trace', 8.924242424242424), ('n-terminal domain', 8.67089817089817), ('murine leukaemia virus', 8.5), ('c-terminal domain', 8.393120393120393), ('major homology region', 8.2), ('mid-line divide', 7.9), ('amino acid type', 7.888888888888889), ('secondary structure element', 7.766210045662101), ('ψ-blast', 7.75), ('predict secondary structure', 7.742400521852577), ('different colour scheme', 7.625), ('capsid protein domain', 7.480090090090091), ('reasonable overall agreement', 7.403846153846153), ('predict β-structure', 7.37573385518591), ('different parameter combination', 7.272727272727273), ('four capsid protein', 7.167777777777779), ('retroviral capsid protein', 7.086969696969698), ('databank search use', 7.080532212885154), ('fv1 gene product', 7.019607843137255), ('multiple structure alignment', 6.95584968530174), ('significant sequence 

In [18]:
## pretty print
for k in rake_keywords:
    print(k)
    for x in rake_keywords[k][:10]:
        print(x)
    print('\n')

BMC_Struct_Biol-4-_-331416
('ψ-blast/quest search strategy', 17.984848484848484)
('small c-terminal domain', 11.643120393120393)
('α-carbon trace', 8.924242424242424)
('n-terminal domain', 8.67089817089817)
('murine leukaemia virus', 8.5)
('c-terminal domain', 8.393120393120393)
('major homology region', 8.2)
('mid-line divide', 7.9)
('amino acid type', 7.888888888888889)
('secondary structure element', 7.766210045662101)


Evid_Based_Complement_Alternat_Med-4-1-1810371
('smoking-induced dna damage', 11.785714285714286)
('green tea polyphenols', 7.351687090817526)
('green tea may', 7.280258519388955)
('green tea exposure', 6.580258519388955)
('dna damage', 5.833333333333334)
('tea polyphenols', 4.919254658385094)
('non-smokers', 4.833333333333334)
('green tea', 4.780258519388955)
('high level', 4.566666666666666)
('human study', 4.52)


Int_J_Cardiovasc_Imaging-4-1-2233708
('spoil gradient-echo image', 13.262295081967213)
('cobalt chromium alloy', 9.0)
('316l stainless steel', 9.0)
('w

In [20]:
from gensim.summarization import keywords as gensim_kw

In [22]:
gensim_keywords = {}

for k in papers:
    text = papers[k]['text']
    gensim_keywords[k] = gensim_kw(text, pos_filter=[], scores=True)

In [23]:
for k in gensim_keywords:
    print(k)
    for x in gensim_keywords[k][:10]:
        print(x)
    print('\n')

BMC_Struct_Biol-4-_-331416
('structure', 0.2975830931287177)
('structural', 0.2975830931287177)
('structurally', 0.2975830931287177)
('sequence similarity', 0.24366900094239735)
('alignment', 0.19872406116422292)
('align', 0.19872406116422292)
('aligns', 0.19872406116422292)
('protein', 0.1497781240478794)
('virus', 0.14691822213700143)
('use', 0.1455418339730745)


Evid_Based_Complement_Alternat_Med-4-1-1810371
('study', 0.3448883869552188)
('green tea', 0.22432232868009983)
('effect', 0.14407360760290838)
('smoking', 0.14286313944524265)
('smoke', 0.14286313944524265)
('cell', 0.14210657837714682)
('lung cancer', 0.130083440209725)
('smoker', 0.12423563260030335)
('use', 0.11717963293809804)
('period', 0.113156333786339)


Int_J_Cardiovasc_Imaging-4-1-2233708
('image quality', 0.24501589171724514)
('artifact', 0.23453198129741426)
('patient', 0.22774694722505312)
('stenting', 0.2209316263257295)
('scan', 0.16987704023061936)
('different', 0.1680633567266794)
('difference', 0.16806335

In [24]:
from summa import keywords as summa_kw

In [25]:
summa_keywords = {}

for k in papers:
    text = papers[k]['text']
    summa_keywords[k] = summa_kw.keywords(
        text,
        additional_stopwords=stopwords.words("english"),
        scores=True
    )

In [27]:
for k in summa_keywords:
    print(k)
    for x in summa_keywords[k][:10]:
        print(x)
    print('\n')

BMC_Struct_Biol-4-_-331416
('structure', 0.29089168075633715)
('structural', 0.29089168075633715)
('structurally', 0.29089168075633715)
('sequence similarity', 0.23033505500434412)
('model base', 0.1904738895083125)
('alignment', 0.1887983963883063)
('align', 0.1887983963883063)
('aligns', 0.1887983963883063)
('fv', 0.17980621480890766)
('use', 0.14381408917822588)


Evid_Based_Complement_Alternat_Med-4-1-1810371
('study', 0.33534373745809015)
('green tea', 0.23274891266654635)
('effect', 0.14144444056678368)
('smoking', 0.13972587939025138)
('smoke', 0.13972587939025138)
('cell', 0.1323260245867326)
('lung cancer', 0.1274541183241642)
('smoker', 0.12207644003039202)
('use', 0.11457727442543961)
('period', 0.111266841164927)


Int_J_Cardiovasc_Imaging-4-1-2233708
('image quality', 0.24460917876922547)
('artifact', 0.22529431702682076)
('patient', 0.2222338565308309)
('stenting', 0.2129109916930292)
('clinically useful', 0.16630707942377376)
('scan', 0.1639989917166909)
('different', 0.

### Часть 5

Сперва хочу посмотреть на поиск без фильтрации по частям речи, а потом уже как пойдет

In [28]:
#lower basic keywords so that it could match
for k in papers:
    raw_list = papers[k]['kw']
    papers[k]['kw'] = [x.lower() for x in raw_list]

In [29]:
def precision(test, standard):
    tp = 0
    fp = 0
    for x in test:
        if x[0] in standard:
            tp += 1
        else:
            fp += 1
    return tp / (tp + fp)

def recall(test, standard):
    tp = 0
    fn = 0
    words = [x[0] for x in test]
    for word in standard:
        if word in words:
            tp += 1
        else:
            fn += 1
    return tp / (tp + fn)

def f_score(test, standard, beta=1):
    p = precision(test, standard)
    r = recall(test, standard)
    if p == 0 and r == 0:
        return 0
    else:
        return (p * r) * (beta * beta + 1) / (beta * beta * p + r)

In [40]:
compDict = {
    'gensim': {
    'f': [],
    'r': [],
    'p': []
},
    'rake': {
    'f': [],
    'r': [],
    'p': []
},
    'summa': {
    'f': [],
    'r': [],
    'p': []
}
}

for k in papers:
    standard = papers[k]['kw']
    
    test_g = gensim_keywords[k]
    test_r = rake_keywords[k]
    test_s = summa_keywords[k]
    
    compDict['gensim']['r'].append(recall(test_g, standard))
    compDict['gensim']['p'].append(precision(test_g, standard))
    compDict['gensim']['f'].append(f_score(test_g, standard))
    compDict['rake']['r'].append(recall(test_r, standard))
    compDict['rake']['p'].append(precision(test_r, standard))
    compDict['rake']['f'].append(f_score(test_r, standard))
    compDict['summa']['r'].append(recall(test_s, standard))
    compDict['summa']['p'].append(precision(test_s, standard))
    compDict['summa']['f'].append(f_score(test_s, standard))
    
#for method in compDict:
#    for metric in compDict[method]:
#        nums = compDict[method][metric]
#        compDict[method][metric] = sum(nums) / len(nums)
        
for method in compDict:
    print(method)
    for metric in compDict[method]:
        L = compDict[method][metric]
        print('%s\t%f' % (metric, sum(L) / len(L)))
        #print(L)
    print('\n')

gensim
f	0.046957
r	0.225873
p	0.026567


rake
f	0.039184
r	0.458016
p	0.020674


summa
f	0.037503
r	0.189365
p	0.021132




Учитывая количество предзаданных ключевых слов и размеры списка, которые предлагаются использованными инструментами, главным показателем будет ``recall``, так как нас интересуют в первую очередь ``true positive`` и ``false negative``.

По этому критерию явным победителем является RAKE, который гораздо лучше улавливает составные фразы (а предзаданные списки состоят в основном из фраз, а не из отдельных слов)

### Часть 4.

Будем считать, что ``RAKE`` достаточно неплохо справился и без POS-фильтрации, поэтому я ее применю только к ``gensim``

In [43]:
gensim_keywords_F = {}

for k in papers:
    text = papers[k]['text']
    gensim_keywords_F[k] = gensim_kw(text, pos_filter=['JJ','NN'], scores=True)
    
compDict['gensim_F'] = {
    'f': [],
    'r': [],
    'p': []    
}    

for k in papers:
    standard = papers[k]['kw']
    
    test_F = gensim_keywords_F[k]
    
    compDict['gensim_F']['r'].append(recall(test_F, standard))
    compDict['gensim_F']['p'].append(precision(test_F, standard))
    compDict['gensim_F']['f'].append(f_score(test_F, standard))


for metric in compDict['gensim_F']:
        L = compDict['gensim_F'][metric]
        print('%s\t%f' % (metric, sum(L) / len(L)))
        #print(L)

f	0.046957
r	0.225873
p	0.026567


Результаты оказались абсолютно идентичны тем, что были

### Часть 6.

На самом деле, сложно сказать, что надо усовершенствовать.

Я вижу несколько проблем:

1. Отсутствие строгих критериев отбора ключевых слов для контроля качества.

Никаких оценок, кроме собственной интуиции и имевшихся заранее тэгов, я дать не смог. При этом имевшиеся тэги говорят не о самых важных и значимых словах, а о некоторых фичах, которые описываются или _косвенно_ затрагиваются в статье.

2. Фильтрация отдельных элементов.

Например, ``RAKE``, хоть и был самым аккуратным из всех инструментов, часто выделял ссылки на другие статьи как ключевые слова (особенно в ``"J_Chem_Ecol-4-1-2292484"``). Также часто попадались цифры и элементы уравнений.